In [1]:
강서구 = [1150010100,1150010200,1150010300,1150010400,1150010500,1150010600,1150010700,1150010800,1150010900,1150011000,1150011100,1150011200,1150011300]
gangseo = ['염창동','등촌동','화곡동','가양동','마곡동','내발산동','외발산동','공항동','방화동','개화동','과해동','오곡동','오쇠동']

In [2]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 강서구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(gangseo)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':강서구,'per price':amean}
df = pd.DataFrame(kk,index = gangseo)
df

,dong,per price
염창동,1150010100,20529583
등촌동,1150010200,50921822
화곡동,1150010300,12632783
가양동,1150010400,19716561
마곡동,1150010500,18237572
내발산동,1150010600,17106826
외발산동,1150010700,0
공항동,1150010800,13832641
방화동,1150010900,15880892
개화동,1150011000,0


In [3]:
df.to_csv('gangseogu.csv')